<html>
<body style="background-image:url(bkg2.png)">
<h1> Python III: Optimization and Heuristics</h1>

<h3> Thank you for joining us. We will be starting shortly.</h3>
</body>
</html>

# Welcome to the webinar

## Python III: Optimization and Heuristics

<html>

<h1>Speaker Introduction</h1>
<div id="container" style="width:100%; float:left;">
<div id="left" style="float:left; width:70%;">
<h3>Dr. Daniel Espinoza</h3>
<ul>
    <li>Senior Developer at Gurobi Optimization<ul>
    <li>Expert in Integer Programming, Cutting planes, Modeling and Software Development</li></ul></li>
    <li>Ph.D. in Operations Research, Georgia Institute of Technology</li>
    <li>Has published several papers on cutting planes, and applications such as Mining, Routing, and Unit Commitment</li>
</ul>
</div><div id="right" style="float:right; width 25%;">
<img src="DE2.jpg" width=160>
</div></div>
</html>

# Python III: Optimization and Heuristics

## Getting and Improving Solutions Quickly!

<html>
<body style="background-image:url(bkg2.png)">
<h1>Summary</h1>
</body>
</html>

<ul>
    <li>The first step</li>
    <li>Controlling termination criteria</li>
    <li>Using greedy heuristics and MIP Starts</li>
    <li>Two simple problems</li>
    <li>Thinning out the set of variables</li>
    <li>Divide and conquer strategies</li>
    <li>Using LP as a guide</li>
</ul>

# The first step

### Get a Tight Model


<ul>
    <li>Root LP value as close as possible to optimal MIP</li>
    <li>Surprisingly, the best model might not be the smallest one</li>
    <li>Consider Cut Callbacks for your problem</li>
    <li>If you can not further improve...</li>
</ul>

# Controlling Termination Criteria and MIP Focus

In [1]:
from gurobipy import *
m = Model()

<ul>
    <li>Controlling MIP focus: 几个聚焦目标
    <ul>
        <li><code>m.setParam('MIPFocus',1)</code>: Focus on finding good solutions 聚焦解</li>  
        <li><code>m.setParam('MIPFocus',2)</code>: Focus on proving optimality 聚焦证明 </li>
        <li><code>m.setParam('MIPFocus',3)</code>: Focus on improving lower bound 聚焦低下限</li>
        <li><code>m.setParam('Heuristics',0.5)</code> Ask Gurobi to spend about 50% of the time looking for improving solutions 花 50% 时间改善结果</li>
    </ul></li>
 </ul>

<ul>
    <li>Setting an acceptable proven gap: We can set both a maximum relative or absolute <span class="blue bold">gap</span>
    <ul>
        <li><code>m.setParam('MIPGap',0.1)</code>: Stop if <span class="blue bold">proven</span> gap is under 10% </li>
        <li><code>m.setParam('MIPGapAbs',1.0)</code>: Stop if <span class="blue bold">proven</span> gap is under 1.0</li>
        <li><code>m.setParam('Cutoff',maxval)</code>: Don't explore nodes whose LP value is above <code>maxval</code></li>
        <li><code>m.setParam('BestObjStop',targetval)</code>: Stop whenever we find a solution with value under <code>targetval</code></li>
        <li><code>m.setParam('BestBdStop',relaxval)</code>: Stop whenever the current bound value is above <code>relaxval</code></li>
    </ul></li>
</ul>

<ul>
    <li>Limiting how much work to perform during a MIP solve:
    <ul>
        <li><code>m.setParam('SolutionLimit',5)</code>: Stop after finding <code>5</code> solutions. The larger the number, the better the quality... but longer to find</li>
        <li><code>m.setParam('NodeLimit',500)</code>: Stop after exploring <code>500</code> nodes in Branch and Bound</li>
        <li><code>m.setParam('IterationLimit',1000000)</code>: Stop after performing <code>1000000</code> simplex iterations</li>
        <li><code>m.setParam('TimeLimit',3600)</code>: Stop after an hour of running time</li>
    </ul></li>
    
</ul>

# Loading Initial Solutions

## A Simple Knapsack Example

In [2]:
from gurobipy import *
import numpy as np

# Build Data
n = 5000  # 50 ~ 50000 都是很好解的
c = {}
w = {}
for i in range(n): # 创建均匀分布的随机变量向量 c (len=50), w
    c[i] = np.random.uniform(low = 1, high = 100)
    w[i] = np.random.uniform(low = 1, high = 100)
b = sum(w)/2


# Build Model
m = Model()
x = m.addVars(n, vtype=GRB.BINARY, name='x') # 添加变量组 gurobi.Var x[0] - x[49]

m.setObjective(x.prod(c), GRB.MAXIMIZE) # 设置目标函数为向量相乘 x * c

m.addConstr(x.prod(w) <= b) # 设置约束函数为向量相乘 x * w

# https://www.gurobi.com/documentation/7.0/refman/mipgap2.html
# mipgap 默认值 	1e-4
m.params.mipgap = 0.01  # 设置 mipgap m.setParam('MIPGap',0.1): Stop if proven gap is under 10%

# https://www.gurobi.com/documentation/7.0/refman/py_var_setattr.html
# 设置 Vars 属性
# 属性清单 https://www.gurobi.com/documentation/7.0/refman/attributes.html#sec:Attributes
# setAddr('Start') # 设置初始值

# Build Dumb Initial Solution
usedb = 0
for i in x:
    # 为何这样作就可以保证可行? (判断数值 小于 <= 均值, 赋值1 放入包中, 如果超过均值, 则不放入包中 0)
    if usedb + w[i] <= b: #ensure feasibility 
        x[i].setAttr('Start',1)
        usedb += w[i]
    else:
        x[i].setAttr('Start',0)

#optimize
m.optimize()

Changed value of parameter mipgap to 0.01
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Optimize a model with 1 rows, 5000 columns and 5000 nonzeros
Variable types: 0 continuous, 5000 integer (5000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+06, 6e+06]
Found heuristic solution: objective 252895

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 252895 
Pool objective bound 252895

Optimal solution found (tolerance 1.00e-02)
Best objective 2.528948415518e+05, best bound 2.528948415518e+05, gap 0.0000%


## When does it help?

<ul>
    <li>When initial solution is hard to come by</li>
    <li>Relaxation is hard to solve</li>
    <li>A quick solution is needed
    <ul>
        <li>Gurobi (given some time) can improve on starting solution</li>
        <li>Helps in pruning search tree</li>
    </ul></li>
</ul>

# Two Example Problems

两个问题 
* TSP
* Open it Mining

# The Traveling Salesman Problem:

### George Dantzig 1954 42-city problem Optimal solution

<img src="dantzig42.png" width="640">

Given a set of <span class="blue bold">cities</span> $N$ to visit, and distances $d_{ij}$ among them, we want to visit them all exactly once, but traveling the smallest distance possible

$$
\begin{array}{lll}
\min & \sum\limits_{i,j in N} d_{ij} x_{ij}\\
s.t. & \sum\limits_{i in N\setminus\{j\}}x_{ij} = 2 &\forall j \in N\\
     & \sum\limits_{i \in S, j\notin S} x_{ij} \geq 2 & \forall \emptyset\neq S\subsetneq N
\end{array}
$$

# Open Pit Mining

<img src="reko2.gif">

## Problem Description and a Mathematical Model

We are given a time-span $T_{max}$ for the mine, and a set of <span class="blue bold">Blocks</span> $\mathcal{B}$ (i.e. pieces) that we could mine.

Our objective is to maximize NPV by scheduling <span class="blue bold">when</span> to extract a given block, while ensuring that the pit don't collapse, and using a maximum processing plant capacity, and available truck capacity.

$$
\begin{array}{lll}
\max & \sum\limits_{b\in \mathcal{B}, t\in\mathcal{T}} c_{b,t} (x_{b,t} - x_{b,t-1})\\
s.t. & \sum\limits_{b\in \mathcal{B}} wp_{b}(x_{b,t}-x_{b,t-1}) \leq Cp_t & \forall t\in\mathcal{T}\\
     & \sum\limits_{b\in \mathcal{B}} wt_{b}(x_{b,t}-x_{b,t-1}) \leq Ct_t & \forall t\in\mathcal{T}\\
     & x_{b,t} \leq x_{b,t+1} & \forall b\in\mathcal{B}, t\in\mathcal{T}\\
     & x_{b,t} \leq x_{b',t} & \forall b\in\mathcal{B}, b'\in\mathrm{above}(b), t\in\mathcal{T}\\
     & x_{b,t} \in \{0,1\} & \forall b\in\mathcal{B}, t\in\mathcal{T}
\end{array}
$$

In this formulation $x_{b,t} = 1$ if block $b$ is extracted in time $t$ or before.
Also, $\mathcal{T} = \{1,\ldots,T_{max}\}$, and $\mathrm{above}(b)$ is the set of <span class="blue bold">precedences</span> of $b$, i.e. those blocks that must be extracted before $b$ can be <span class="blue bold">safely</span> extracted.

# MIP Starts

In [3]:
# 禁止 matplotlib inline , 避免嵌入在页面中, 增加冗余代码
%matplotlib

Using matplotlib backend: MacOSX


In [4]:
run tsp2.py 10 -D


Solution cost: 348.198 gap 0.00%

Total running time: 0.33


In [5]:
run tsp2.py 200 -v -T 10

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter timelimit to 9.57497596741
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 200 rows, 19900 columns and 39800 nonzeros
Variable types: 0 continuous, 19900 integer (19900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.03s
Presolved: 200 rows, 19900 columns, 39800 nonzeros
Variable types: 0 continuous, 19900 integer (19900 binary)
Found heuristic solution: objective 10680.755769

Root relaxation: objective 9.819618e+02, 309 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  981.96181    0   58 10680.7558  981.96181  90.8%     -    0s
     0     0 1001.16570    0   46 10680.7

# Mip Starts

A classical <span class="blue bold">construction</span> heuristic for TSP is:
<ul>
    <li>Label all nodes as <span class="blue bold">unvisited</span></li>
    <li>Start from a given node</li>
    <li>Mark node as visited</li>
    <li>Select next node as closest node to current node</li>
    <li>Set current node as new node, <code>repeat</code></li>
</ul>

-G 参数: 贪婪启发式算法
run'-G','--greedy', help='Enable initial greedy heuristic'

In [6]:
run tsp2.py 150 -G -v -T 10 -D

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0

----------------------------------------------------------------------
Found feasible solution, value 1232.35 0.14 seconds
----------------------------------------------------------------------

Changed value of parameter timelimit to 9.62461185455
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 150 rows, 11175 columns and 22350 nonzeros
Variable types: 0 continuous, 11175 integer (11175 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve time: 0.03s
Presolved: 150 rows, 11175 columns, 22350 nonzeros

Loaded MIP start with objective 1232.35

Variable types: 0 continuous, 11175 integer (11175 binary)

Root relaxation: objective 8.534270e+02, 229 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl

Note that we went from no solution in 10 seconds, to an optimal solution in five seconds!

# MIP Starts

For the case of Open Pit Mining... we can also think of a greedy heuristic
<ul>
    <li>Starting from time zero</li>
    <li>Choose the <span class="blue bold">best</span> block that can be mined and processed in the current period</li>
    <li>Update remaining capacities</li>
    <li>Repeat until no block can be fit</li>
    <li>Move to next time period, and reset remaining capacities</li>
    <li>Repeat until last time period</li>
</ul>

In [7]:
run mining.py -Z15 -T8 -v

----------------------------------------------------------------------
Building grid blocks ... 
done (2475 added)
Add variables and objective function ... 
done (added 27225 vars obj range [-2.07482,4.13126])
Adding precedences ... 
done (added 141900 precedences)
Adding knapsack capacities ... 
done (added 22 knapsack)
Changed value of parameter timelimit to 3.56007599831
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
----------------------------------------------------------------------
Model Stats:

Statistics for model Unnamed :
  Linear constraint matrix    : 141922 Constrs, 27225 Vars, 349236 NZs
  Variable types              : 0 Continuous, 27225 Integer (27225 Binary)
  Matrix coefficient range    : [ 0.900027, 1.09997 ]
  Objective coefficient range : [ 4.04706e-05, 1.02118 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 85.2273, 102.273 ]
Model bound: 100000000000000001590289110975991804683608085639452813897813275577478387721703810608134

## Using Greedy

What if we enable our greedy heuristic?

-G 参数: 贪婪启发式算法 run'-G','--greedy', help='Enable initial greedy heuristic'

In [8]:
run mining.py -Z15 -T8 -Gv

----------------------------------------------------------------------
Building grid blocks ... 
done (2475 added)
Add variables and objective function ... 
done (added 27225 vars obj range [-2.07482,4.13126])
Adding precedences ... 
done (added 141900 precedences)
Adding knapsack capacities ... 
done (added 22 knapsack)
Building solution ... 

----------------------------------------------------------------------
Greedy Heuristic done, value 191.178
----------------------------------------------------------------------

Changed value of parameter timelimit to 3.59042596817
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
----------------------------------------------------------------------
Model Stats:

Statistics for model Unnamed :
  Linear constraint matrix    : 141922 Constrs, 27225 Vars, 349236 NZs
  Variable types              : 0 Continuous, 27225 Integer (27225 Binary)
  Matrix coefficient range    : [ 0.900027, 1.09997 ]
  Objective coefficient range : [ 4.04706e-05, 


# What happened?

<ul>
    <li>The first Gurobi solution is very weak for this problem... it starts at the last time period</li>
    <li>Our greedy heuristic seems to be using <span class="blue bold">useless</span> blocks</li>
</ul>

<ul>
    <li>Shouldn't Gurobi somehow handle that?
    <ul>
        <li>Pre-processing helps on that</li>
        <li>Having an good LP-relaxation also helps</li>
        <li>...But it's not magic</li>
        <li>If you know something about your variables (and the problem is getting hard)... you should use it</li>
    </ul></li>
</ul>

# Thinning out the set of variables

<ul>
    <li>In the case of TSP:
    <ul>
        <li><span class="blue bold">Probably</span> edges that are too long will not be part of any optimal solution</li>
        <li><code>for x in vars.keys() if vars[x].getAttr('Obj') > max_dist:
  vars[x].setAttr('Ub',0)</code>
        <li>Must guess a reasonable value... (but LP can help, use reduced costs)</li>
    </ul></li>
</ul>


<ul>
    <li>In the case of Open Pit Mining:
    <ul>
        <li>By the nature of the problem, many blocks are just filling in</li>
        <li>e.g. If a block is at the <span class="blue bold">bottom</span> of the mine, and has a negative value, you will <span class="blue bold">never</span> mine it</li>
        <li>The previous procedure can be done recursively</li>
        <li>Use other knowledge:
        <ul>
            <li>By integrality, to mine a block, blocks above it must also be mined</li>
            <li>This induces a minimum extraction time</li>
        </ul></li>
    </ul></li>
</ul>

<ul>
    <li>Gurobi does a lot... but</li>
</ul>

In [9]:
run mining.py -Z15 -T8 -O0

----------------------------------------------------------------------
Building grid blocks ... 
done (2475 added)
Add variables and objective function ... 
done (added 27225 vars obj range [-2.07482,4.13126])
Adding precedences ... 
done (added 141900 precedences)
Adding knapsack capacities ... 
done (added 22 knapsack)
Changed value of parameter timelimit to 2.58489704132
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
----------------------------------------------------------------------
Model Stats:

Statistics for model Unnamed :
  Linear constraint matrix    : 141922 Constrs, 27225 Vars, 349236 NZs
  Variable types              : 0 Continuous, 27225 Integer (27225 Binary)
  Matrix coefficient range    : [ 0.900027, 1.09997 ]
  Objective coefficient range : [ 4.04706e-05, 1.02118 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 85.2273, 102.273 ]
Problem not optimized
Total Running time: 5.49


In [10]:
run mining.py -Z15 -T8 -O0 -P

----------------------------------------------------------------------
Building grid blocks ... 
done (2475 added)
Preprocessing removed 1556 blocks 17116 vars
Remove 248 time-blocks
Add variables and objective function ... 
done (added 9861 vars obj range [-0.574192,4.13126])
Adding precedences ... 
done (added 46554 precedences)
Adding knapsack capacities ... 
done (added 22 knapsack)
Changed value of parameter timelimit to 5.77027392387
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
----------------------------------------------------------------------
Model Stats:

Statistics for model Unnamed :
  Linear constraint matrix    : 46576 Constrs, 9861 Vars, 125016 NZs
  Variable types              : 0 Continuous, 9861 Integer (9861 Binary)
  Matrix coefficient range    : [ 0.900027, 1.09997 ]
  Objective coefficient range : [ 4.04706e-05, 1.02118 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 85.2273, 102.273 ]
Problem not optimized
Total Running t

We went from <span class="red">141,922</span> constraints and <span class="red">27,225</span> variables to <span class="blue">46,586</span> constraints and <span class="blue">9,924</span> variables!

And what happen now with our greedy heuristic?

In [11]:
run mining.py -Z15 -T8 -PGv

----------------------------------------------------------------------
Building grid blocks ... 
done (2475 added)
Preprocessing removed 1556 blocks 17116 vars
Remove 248 time-blocks
Add variables and objective function ... 
done (added 9861 vars obj range [-0.574192,4.13126])
Adding precedences ... 
done (added 46554 precedences)
Adding knapsack capacities ... 
done (added 22 knapsack)
Building solution ... 

----------------------------------------------------------------------
Greedy Heuristic done, value 195.252
----------------------------------------------------------------------

Changed value of parameter timelimit to 5.71937298775
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
----------------------------------------------------------------------
Model Stats:

Statistics for model Unnamed :
  Linear constraint matrix    : 46576 Constrs, 9861 Vars, 125016 NZs
  Variable types              : 0 Continuous, 9861 Integer (9861 Binary)
  Matrix coefficient range    : [ 0.90

TypeError: list indices must be integers, not float

# Divide and Conquer Strategies

## AKA MIP in Local Search

Why would it make sense?
<ul>
    <li>Problems get harder as they grow, the reverse is also true</li>
    <li>If problem has natural <span class="blue bold">pieces</span> we can search in parallel!</li>
</ul>

<ul>
    <li>In the case of the TSP:<ul>
        <li>Each tour segment, of any length, can be used as a partition</li>
        <li>Can work on <span class="blue bold">geometric</span> partitions of the points too</li>
    </ul></li>
</ul>

<ul>
    <li>In the case of the Open Pit:<ul>
        <li>Each time period is a natural partition</li>
        <li>Can use groups of consecutive time periods</li>
    </ul></li>
</ul>

<ul>
    <li>In unit commitment/machine scheduling you can partition by machine and/or time</li>
    <li>We only need a feasible solution <code>Xo</code> and a <code>subset</code> of the variables:
    <code>
for x in vars.keys() if not in subset:
    vars[x].setAttr('LB',Xo[x])
    vars[x].setAttr('UB',Xo[x])
    </code>
    </li>
    <li>And then, optimize resulting problem</li>
</ul>

# Divide and Conquer Strategies

## AKA MIP in Local Search

In [12]:
run tsp2.py 100 -GI -S5 -D -O0


----------------------------------------------------------------------
Found feasible solution, value 1060.8 0.07 seconds
----------------------------------------------------------------------

Improve solution 1059.87 0.30 seconds
Improve solution 1059.85 1.07 seconds
Improve solution 1055.87 1.26 seconds
Improve solution 1049.54 1.44 seconds
Improve solution 1049.07 2.09 seconds
Improve solution 1046.09 2.29 seconds
Improve solution 1039.89 2.62 seconds
Improve solution 1032.54 2.93 seconds
Improve solution 1024.73 3.20 seconds
Improve solution 1015.4 3.40 seconds
Improve solution 1010.85 3.60 seconds
Improve solution 1004.52 3.80 seconds
Improve solution 1004.21 9.45 seconds
Improve solution 1001.22 9.63 seconds
Improve solution 1000.83 11.81 seconds

----------------------------------------------------------------------
Final solution 1000.83 subproblems 100 optimal 100 timelimit 0 12.54 seconds
----------------------------------------------------------------------

No solution fo

In [13]:
run mining.py -Z15 -PGIvv -O0

----------------------------------------------------------------------
Building grid blocks ... 
done (2475 added)
Preprocessing removed 1556 blocks 17116 vars
Remove 248 time-blocks
Add variables and objective function ... 
done (added 9861 vars obj range [-0.574192,4.13126])
Adding precedences ... 
done (added 46554 precedences)
Adding knapsack capacities ... 
done (added 22 knapsack)
Building solution ... 

Moving to time 1 obj 69.1699 capacity 83.19% 99.83% 
Moving to time 2 obj 114.762 capacity 83.10% 99.71% 
Moving to time 3 obj 145.983 capacity 82.73% 99.28% 
Moving to time 4 obj 169.799 capacity 83.05% 99.66% 
Moving to time 5 obj 183.388 capacity 83.19% 99.82% 
Moving to time 6 obj 197.575 capacity 82.80% 99.36% 
Moving to time 7 obj 199.41 capacity 99.53% 56.11% 
Moving to time 8 obj 204.316 capacity 99.40% 45.01% 
Moving to time 9 obj 202.204 capacity 99.26% 28.41% 
Moving to time 10 obj 195.485 capacity 99.84% 24.07% 

-------------------------------------------------------

# Divide and Conquer Strategies

Note that in any local search heuristic, we have a controlling parameter, which handles the size of the <span class="blue bold">neighborhood</span> where we will loop for better solutions.

<ul>
    <li>In the case of the TSP: The length of the sub-segment to optimize</li>
    <li>Or the number of cities in a region to consider</li>
    <li>In the case of Open Pit Mining: The number of consecutive time periods to re-optimize</li>
</ul>

### The Trade Off
<ul>
    <li>The larger the neighborhood, the better the final solution</li>
    <li>The smaller the neighborhood, the faster the optimization process</li>
</ul>

# Divide and Conquer Strategies

### In the TSP:

In [14]:
run tsp2.py 100 -GI -S5 -O0


----------------------------------------------------------------------
Found feasible solution, value 1060.8 0.05 seconds
----------------------------------------------------------------------

Improve solution 1059.87 0.22 seconds
Improve solution 1059.85 0.87 seconds
Improve solution 1055.87 1.04 seconds
Improve solution 1049.54 1.20 seconds
Improve solution 1049.07 1.79 seconds
Improve solution 1046.09 1.96 seconds
Improve solution 1039.89 2.27 seconds
Improve solution 1032.54 2.59 seconds
Improve solution 1024.73 2.92 seconds
Improve solution 1015.4 3.17 seconds
Improve solution 1010.85 3.40 seconds
Improve solution 1004.52 3.59 seconds
Improve solution 1004.21 9.32 seconds
Improve solution 1001.22 9.53 seconds
Improve solution 1000.83 11.90 seconds

----------------------------------------------------------------------
Final solution 1000.83 subproblems 100 optimal 100 timelimit 0 12.68 seconds
----------------------------------------------------------------------

No solution fo

In [15]:
run tsp2.py 100 -GI -S10 -O0


----------------------------------------------------------------------
Found feasible solution, value 1060.8 0.06 seconds
----------------------------------------------------------------------

Improve solution 1050.44 0.15 seconds
Improve solution 1048.05 0.40 seconds
Improve solution 1021.28 1.00 seconds
Improve solution 1019.92 1.17 seconds
Improve solution 1012.56 1.62 seconds
Improve solution 1010.71 1.82 seconds
Improve solution 1010.54 2.14 seconds
Improve solution 995.614 2.32 seconds
Improve solution 993.228 2.49 seconds
Improve solution 984.927 2.71 seconds
Improve solution 981.427 2.94 seconds
Improve solution 961.286 3.27 seconds
Improve solution 958.964 3.45 seconds
Improve solution 927.89 3.64 seconds
Improve solution 925.953 3.82 seconds
Improve solution 924.205 4.02 seconds
Improve solution 914.534 4.24 seconds
Improve solution 911.236 6.40 seconds
Improve solution 910.188 12.84 seconds
Improve solution 903.894 13.46 seconds
Improve solution 889.488 13.63 seconds

----

### Something similar in the Open Pit Problem

In [16]:
run mining.py -PGIvv -O0 -S2 -Z20

----------------------------------------------------------------------
Building grid blocks ... 
done (6000 added)
Preprocessing removed 3621 blocks 54315 vars
Remove 1008 time-blocks
Add variables and objective function ... 
done (added 34677 vars obj range [-0.534185,4.17165])
Adding precedences ... 
done (added 174408 precedences)
Adding knapsack capacities ... 
done (added 30 knapsack)
Building solution ... 

Moving to time 1 obj 137.805 capacity 83.31% 99.97% 
Moving to time 2 obj 231.252 capacity 83.16% 99.79% 
Moving to time 3 obj 306.817 capacity 83.31% 99.97% 
Moving to time 4 obj 365.266 capacity 82.93% 99.52% 
Moving to time 5 obj 413.569 capacity 82.99% 99.59% 
Moving to time 6 obj 440.345 capacity 82.94% 99.53% 
Moving to time 7 obj 470.045 capacity 83.14% 99.77% 
Moving to time 8 obj 487.355 capacity 83.08% 99.69% 
Moving to time 9 obj 498.644 capacity 82.90% 99.48% 
Moving to time 10 obj 506.472 capacity 85.26% 99.66% 
Moving to time 11 obj 512.113 capacity 99.74% 47.46%

In [17]:
run mining.py -PGIvv -Z20 -S4 -O0

----------------------------------------------------------------------
Building grid blocks ... 
done (6000 added)
Preprocessing removed 3621 blocks 54315 vars
Remove 1008 time-blocks
Add variables and objective function ... 
done (added 34677 vars obj range [-0.534185,4.17165])
Adding precedences ... 
done (added 174408 precedences)
Adding knapsack capacities ... 
done (added 30 knapsack)
Building solution ... 

Moving to time 1 obj 137.805 capacity 83.31% 99.97% 
Moving to time 2 obj 231.252 capacity 83.16% 99.79% 
Moving to time 3 obj 306.817 capacity 83.31% 99.97% 
Moving to time 4 obj 365.266 capacity 82.93% 99.52% 
Moving to time 5 obj 413.569 capacity 82.99% 99.59% 
Moving to time 6 obj 440.345 capacity 82.94% 99.53% 
Moving to time 7 obj 470.045 capacity 83.14% 99.77% 
Moving to time 8 obj 487.355 capacity 83.08% 99.69% 
Moving to time 9 obj 498.644 capacity 82.90% 99.48% 
Moving to time 10 obj 506.472 capacity 85.26% 99.66% 
Moving to time 11 obj 512.113 capacity 99.74% 47.46%

# Using LP as a guide

Using LP to guide a MIP search:
<ul>
    <li>Complexity in solving MIP mainly comes from binary/integer variables</li>
    <li>Before we tried <span class="blue bold">fixing</span> part of the problem and optimizing the remainder</li>
    <li>We can also <span class="blue bold">relax</span> the integrality requirements for part of the problem</li>
</ul>

<ul>
    <li>The hope is that the continuous relaxation conveys some information when deciding integer variables</li>
    <li>How to do it?
<code>
for x in vars.keys() if x in relaxset:
    vars[x].setAttr('VType',GRB.CONTINUOUS)
</code></li>
</ul>

<ul>
    <li>In the Open Pit Problem:<ul>
        <li>We can relax integrality of variables with $T\geq1$, and optimize</li>
        <li>Fix solution for $T=0$, add integrality requirements for $T=1$, and optimize</li>
        <li>... Repeat until $T_{max}$</li>
        </ul></li>
</ul>

<ul>
    <li>We can use fractional solution to perform a <span class="blue bold">guided</span> rounding</li>
    <li>In this form, we can use the heuristic as a <span class="blue bold">MIP-node callback</span></li>
    <li>Gurobi will call your code in some nodes, and accept any user-provided solution</li>
</ul>

<ul>
     <li>The previous scheme is known as <span class="blue bold">Rolling Horizon</span> heuristic</li>
     <li>We can keep integrality requirements for $k_1$ periods, and fix only $k_2\leq k_1$ periods</li>
     <li>We can <span class="blue bold">disregards</span> periods over $k_2$
<code>
for x in Vars.keys():
    if time[x] > k_2:
        Vars[x].lb = 0
        Vars[x].ub = 0
</code>
     </li>
     <li>It can bee seen as construction heuristic</li>
     <li>It can also be used as a (expensive) callback heuristic</li>
</ul>

<ul>
    <li>When it helps?
    <ul>
        <li>When the LP relaxation bound is <span class="blue bold">close</span> to the optimal integer solution</li>
        <li>i.e. A tighter formulation will produce better results</li>
        <li>The Open Pit Problem is one such example</li>
        <li>TSP can be an example.... requires sub-tour as callback cut, not lazy cut</li>
    </ul></li>
</ul>

# Using LP as a guide

### In the Open Pit problem:

In [18]:
run mining.py -PRvv -O0 -Z20 -S1

----------------------------------------------------------------------
Building grid blocks ... 
done (6000 added)
Preprocessing removed 3621 blocks 54315 vars
Remove 1008 time-blocks
Add variables and objective function ... 
done (added 34677 vars obj range [-0.534185,4.17165])
Adding precedences ... 
done (added 174408 precedences)
Adding knapsack capacities ... 
done (added 30 knapsack)
Changed value of parameter presolve to 1
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter method to 2
   Prev: -1  Min: -1  Max: 4  Default: -1
Changed value of parameter mipgap to 0.01
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001


TypeError: range() integer start argument expected, got float.

# More on the scripts:

### Will be available for download soon

In [19]:
run tsp2.py -h

usage: tsp2.py [-h] [-G] [-I] [-D] [-v] [-S SUBMIPSIZE] [-O OPTIMIZE]
               [-T TIMELIMIT]
               cities

positional arguments:
  cities                Number of cities in TSP

optional arguments:
  -h, --help            show this help message and exit
  -G, --greedy          Enable initial greedy heuristic (default: 0)
  -I, --improve         Improve initial greedy heuristic with sub-mips
                        (default: 0)
  -D, --drawsolution    Draw final mip solution (default: 0)
  -v, --verbose         verbose output (default: 0)
  -S SUBMIPSIZE, --submipsize SUBMIPSIZE
                        Number of cities to consider in sub-mip (default: 10)
  -O OPTIMIZE, --optimize OPTIMIZE
                        Perform full problem optimization (default: 1)
  -T TIMELIMIT, --timelimit TIMELIMIT
                        Time limit for optimization (default: 1e+100)


In [20]:
run mining.py -h

usage: mining.py [-h] [-C] [-G] [-I] [-O OPTIMIZE] [-P] [-R] [-S SUBMIPSIZE]
                 [-T TIMELIMIT] [-v] [-Z MINESIZE]

optional arguments:
  -h, --help            show this help message and exit
  -C, --callback        Use Heuristic as Callbacks (default: 0)
  -G, --greedy          Enable initial greedy heuristic (default: 0)
  -I, --improve         Improve initial greedy heuristic with sub-mips
                        (default: 0)
  -O OPTIMIZE, --optimize OPTIMIZE
                        Perform full problem optimization (default: 1)
  -P, --preprocess      Preprocess variables (default: 0)
  -R, --rollinghorizon  Enable initial rolling horizon heuristic (default: 0)
  -S SUBMIPSIZE, --submipsize SUBMIPSIZE
                        Periods in Time-Window heuristic (default: 3)
  -T TIMELIMIT, --timelimit TIMELIMIT
                        Time limit for optimization (default: 1e+100)
  -v, --verbose         verbose output (default: 0)
  -Z MINESIZE, --minesize MINESIZE
      

# Questions?

#### Please submit your questions using the questions area on the right of your screen

# Thank you for joining us

<ul>
<li>If you haven’t already done so, please register at http://www.gurobi.com, and then visit http://www.gurobi.com/get-anaconda to try Gurobi and Python for yourself.</li>

<li>Look for information on our next Python webinar, Machine Learning and Optimization.</li>

<li>For questions about pricing, please contact <span class="blue bold">sales@gurobi.com</span> or <span class="blue bold">sales@gurobi.de</span>.</li>

<li>A recording of the webinar, including the slides, will be available in roughly one week.</li>
</ul>